In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
from datetime import *
from sklearn import linear_model
from sklearn.preprocessing import *
from sklearn import ensemble
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import xgboost as xgb
from sklearn import metrics
from xgboost.sklearn import XGBClassifier  
from sklearn.model_selection import GridSearchCV,cross_val_score  
import matplotlib.pylab as plt  

In [5]:
#首先导入滑窗训练集数据进行线下数据集划分
train_df = pd.read_csv('chusai_huachuang_train_df.csv')
test_df = pd.read_csv('chusai_huachuang_test_df.csv')

fea = [
#         'context_page_id_1', 'context_page_id_2', 'context_page_id_3', 'context_page_id_4', 
       'item_pv_level_low', 'shop_star_level_loss', 'user_gender_id', 'user_occupation_id', 'user_star_level',
       'hour', 'day_hour', #'weekday',
       'user_age_level', 'item_sales_level', 'item_brand_id', 'item_city_id',
       'item_collected_level', 'item_price_level', 'item_pv_level',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
#        'match_category_proportion', 'match_property_proportion',
#        'predict_category_number', 'predict_property_number', 
        'category_number', 'property_number', 
#        'isFirstCategoryIn', 'isLastCategoryIn', 
       
        'lastOneHour_sameItem_count', 'lastOneHour_sameFirstCategory_count', 'lastOneHour_sameLastCategory_count', 
       'lastOneHour_sameBrand_count', 'lastOneHour_sameShop_count', 
    
        'lastOneDay_sameItem_count', 'lastOneDay_sameFirstCategory_count', 'lastOneDay_sameLastCategory_count', 
       'lastOneDay_sameBrand_count', 'lastOneDay_sameShop_count', 
        
#        'isLastOneHour_firstClickItem', 
       'is_history_sameItem', 'is_history_sameFirstCategory', 'is_history_sameLastCategory',
       'is_highProportion_brand', 'is_highSale_brand', 
       'isHigh_shop_score_service', 'isHigh_shop_score_delivery',
       'isHigh_shop_score_description', 'isHigh_shop_review_positive_rate',
#        'is_history_sameBrand', 'is_history_sameShop', #'is_history_firstClickItem',
#        'shop_review_positive_rate_diff', 'shop_score_service_diff', 'shop_score_delivery_diff', 'shop_score_description_diff', 
        'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
#        'shop_item_classNumber', 'brand_item_classNumber', 'city_item_classNumber', 
#        'shop_user_classNumber', 'brand_user_classNumber', 'city_user_classNumber',
       
        'hour_converse_smooth_rate',
    
 
    
#         'user_id_total_number', 
#     'item_brand_id_total_number', 'item_id_total_number', 
#        'shop_id_total_number', 'item_city_id_total_number', 
#        'real_first_category_total_number', 'real_last_category_total_number', 
#        'user_item_total_number', 'user_brand_total_number', 'user_shop_total_number',
#         'user_city_total_number', 'user_first_category_total_number', 'user_last_category_total_number',
        
#         'user_id_buy_number', 
#     'item_brand_id_buy_number', 'item_id_buy_number', 
#        'shop_id_buy_number', 'item_city_id_buy_number', 
#        'real_first_category_buy_number', 'real_last_category_buy_number', 
#        'user_item_buy_number', 'user_brand_buy_number', 'user_shop_buy_number',
#         'user_city_buy_number', 'user_first_category_buy_number', 'user_last_category_buy_number', 
        
        'user_id_converse_smooth_rate', 'item_brand_id_converse_smooth_rate', 'item_id_converse_smooth_rate', 
#        'shop_id_converse_smooth_rate', 'item_city_id_converse_smooth_rate', 
#        'real_first_category_converse_smooth_rate', 'real_last_category_converse_smooth_rate', 
#        'user_item_converse_smooth_rate', 'user_brand_converse_smooth_rate', 'user_shop_converse_smooth_rate',
#         'user_city_converse_smooth_rate', 'user_first_category_converse_smooth_rate', 'user_last_category_converse_smooth_rate', 
       
#         'user_item_price_converse_smooth_rate', 'user_item_sales_converse_smooth_rate', 
#         'user_item_price_total_number', 'user_item_sales_total_number',
#         'user_item_price_buy_number', 'user_item_sales_buy_number',
        
#         'userItem_lastClickDeltaTime', 'userBrand_lastClickDeltaTime', 'userShop_lastClickDeltaTime', 
#         'userFirstCategory_lastClickDeltaTime', 'userLastCategory_lastClickDeltaTime', 
    
#     'user_gender_item_converse_smooth_rate', 
    'user_gender_brand_converse_smooth_rate', 'user_gender_shop_converse_smooth_rate', 
#     'user_age_item_converse_smooth_rate', 'user_age_brand_converse_smooth_rate', 'user_age_shop_converse_smooth_rate', 
#     'user_occupation_item_converse_smooth_rate', 'user_occupation_brand_converse_smooth_rate', 'user_occupation_shop_converse_smooth_rate', 
#     'user_star_item_converse_smooth_rate', 'user_star_brand_converse_smooth_rate', 'user_star_shop_converse_smooth_rate', 
    
#     'user_gender_item_total_number', 'user_gender_brand_total_number', 'user_gender_shop_total_number', 
#     'user_age_item_total_number', 'user_age_brand_total_number', 'user_age_shop_total_number', 
#     'user_occupation_item_total_number', 'user_occupation_brand_total_number', 'user_occupation_shop_total_number',
#     'user_star_item_total_number', 'user_star_brand_total_number', 'user_star_shop_total_number', 
    
#      'user_gender_item_buy_number', 'user_gender_brand_buy_number', 'user_gender_shop_buy_number', 
#       'user_age_item_buy_number', 'user_age_brand_buy_number', 'user_age_shop_buy_number', 
#     'user_occupation_item_buy_number', 'user_occupation_brand_buy_number', 'user_occupation_shop_buy_number',
#     'user_star_item_buy_number', 'user_star_brand_buy_number',  'user_star_shop_buy_number',
    
        'history_sameItem_count', 'history_sameFirstCategory_count', 'history_sameLastCategory_count',
        'history_sameBrand_count', 'history_sameShop_count', 
    
                'is_later_clickSameItem', 'is_later_clickSameLastCategory',  
        'later_clickSameItem_count', 'later_clickSameLastCategory_count'
    
      ]

train_dataset1_x = train_df[fea][(train_df.day >= 21) & (train_df.day <= 23)]
train_dataset1_y = train_df['is_trade'][(train_df.day >= 21) & (train_df.day <= 23)]
test_dataset1_x = train_df[fea][(train_df.day == 24)]
test_dataset1_y = train_df['is_trade'][(train_df.day == 24)]

train_dataset_x = train_df[fea][(train_df.day >= 21) & (train_df.day <= 24)]
train_dataset_y = train_df['is_trade'][(train_df.day >= 21) & (train_df.day <= 24)]
test_dataset_x = test_df[fea]
print(len(train_dataset_x.columns))
print(len(test_dataset_x.columns))


57
57


In [6]:
# train_df = pd.read_csv('chusai_huachuang_train_df.csv')
# train_df1 = pd.read_csv('chusai_huachuang_train_df1.csv')
# # test_df = pd.read_csv('chusai_huachuang_test_df.csv')

# fea = [
# #         'context_page_id_1', 'context_page_id_2', 'context_page_id_3', 'context_page_id_4', 
#        'item_pv_level_low', 'shop_star_level_loss', 'user_gender_id', 'user_occupation_id', 'user_star_level',
#        'hour', 'day_hour', #'weekday',
#        'user_age_level', 'item_sales_level', 'item_brand_id', 'item_city_id',
#        'item_collected_level', 'item_price_level', 'item_pv_level',
#        'context_page_id', 'shop_review_num_level', 'shop_star_level',
# #        'match_category_proportion', 'match_property_proportion',
# #        'predict_category_number', 'predict_property_number', 
#         'category_number', 'property_number', 
# #        'isFirstCategoryIn', 'isLastCategoryIn', 
       
#         'lastOneHour_sameItem_count', 'lastOneHour_sameFirstCategory_count', 'lastOneHour_sameLastCategory_count', 
#        'lastOneHour_sameBrand_count', 'lastOneHour_sameShop_count', 
    
#         'lastOneDay_sameItem_count', 'lastOneDay_sameFirstCategory_count', 'lastOneDay_sameLastCategory_count', 
#        'lastOneDay_sameBrand_count', 'lastOneDay_sameShop_count', 
        
# #        'isLastOneHour_firstClickItem', 
#        'is_history_sameItem', 'is_history_sameFirstCategory', 'is_history_sameLastCategory',
#        'is_highProportion_brand', 'is_highSale_brand', 
#        'isHigh_shop_score_service', 'isHigh_shop_score_delivery',
#        'isHigh_shop_score_description', 'isHigh_shop_review_positive_rate',
# #        'is_history_sameBrand', 'is_history_sameShop', #'is_history_firstClickItem',
# #        'shop_review_positive_rate_diff', 'shop_score_service_diff', 'shop_score_delivery_diff', 'shop_score_description_diff', 
#         'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
# #        'shop_item_classNumber', 'brand_item_classNumber', 'city_item_classNumber', 
# #        'shop_user_classNumber', 'brand_user_classNumber', 'city_user_classNumber',
       
#         'hour_converse_smooth_rate',
    
 
    
# #         'user_id_total_number', 
# #     'item_brand_id_total_number', 'item_id_total_number', 
# #        'shop_id_total_number', 'item_city_id_total_number', 
# #        'real_first_category_total_number', 'real_last_category_total_number', 
# #        'user_item_total_number', 'user_brand_total_number', 'user_shop_total_number',
# #         'user_city_total_number', 'user_first_category_total_number', 'user_last_category_total_number',
        
# #         'user_id_buy_number', 
# #     'item_brand_id_buy_number', 'item_id_buy_number', 
# #        'shop_id_buy_number', 'item_city_id_buy_number', 
# #        'real_first_category_buy_number', 'real_last_category_buy_number', 
# #        'user_item_buy_number', 'user_brand_buy_number', 'user_shop_buy_number',
# #         'user_city_buy_number', 'user_first_category_buy_number', 'user_last_category_buy_number', 
        
#         'user_id_converse_smooth_rate', 'item_brand_id_converse_smooth_rate', 'item_id_converse_smooth_rate', 
# #        'shop_id_converse_smooth_rate', 'item_city_id_converse_smooth_rate', 
# #        'real_first_category_converse_smooth_rate', 'real_last_category_converse_smooth_rate', 
# #        'user_item_converse_smooth_rate', 'user_brand_converse_smooth_rate', 'user_shop_converse_smooth_rate',
# #         'user_city_converse_smooth_rate', 'user_first_category_converse_smooth_rate', 'user_last_category_converse_smooth_rate', 
       
# #         'user_item_price_converse_smooth_rate', 'user_item_sales_converse_smooth_rate', 
# #         'user_item_price_total_number', 'user_item_sales_total_number',
# #         'user_item_price_buy_number', 'user_item_sales_buy_number',
        
# #         'userItem_lastClickDeltaTime', 'userBrand_lastClickDeltaTime', 'userShop_lastClickDeltaTime', 
# #         'userFirstCategory_lastClickDeltaTime', 'userLastCategory_lastClickDeltaTime', 
    
# #     'user_gender_item_converse_smooth_rate', 
#     'user_gender_brand_converse_smooth_rate', 'user_gender_shop_converse_smooth_rate', 
# #     'user_age_item_converse_smooth_rate', 'user_age_brand_converse_smooth_rate', 'user_age_shop_converse_smooth_rate', 
# #     'user_occupation_item_converse_smooth_rate', 'user_occupation_brand_converse_smooth_rate', 'user_occupation_shop_converse_smooth_rate', 
# #     'user_star_item_converse_smooth_rate', 'user_star_brand_converse_smooth_rate', 'user_star_shop_converse_smooth_rate', 
    
# #     'user_gender_item_total_number', 'user_gender_brand_total_number', 'user_gender_shop_total_number', 
# #     'user_age_item_total_number', 'user_age_brand_total_number', 'user_age_shop_total_number', 
# #     'user_occupation_item_total_number', 'user_occupation_brand_total_number', 'user_occupation_shop_total_number',
# #     'user_star_item_total_number', 'user_star_brand_total_number', 'user_star_shop_total_number', 
    
# #      'user_gender_item_buy_number', 'user_gender_brand_buy_number', 'user_gender_shop_buy_number', 
# #       'user_age_item_buy_number', 'user_age_brand_buy_number', 'user_age_shop_buy_number', 
# #     'user_occupation_item_buy_number', 'user_occupation_brand_buy_number', 'user_occupation_shop_buy_number',
# #     'user_star_item_buy_number', 'user_star_brand_buy_number',  'user_star_shop_buy_number',
    
#         'history_sameItem_count', 'history_sameFirstCategory_count', 'history_sameLastCategory_count',
#         'history_sameBrand_count', 'history_sameShop_count', 
    
#                 'is_later_clickSameItem', 'is_later_clickSameLastCategory',  
# #         'later_clickSameItem_count', 'later_clickSameLastCategory_count'
    
#       ]

# print(train_df[fea].info())
# print(train_df1[fea].info())
# print(train_df[fea].values == train_df1[fea].values)

In [13]:
#开始进行xgb模型训练
xgb_train_dataset1 = xgb.DMatrix(train_dataset1_x,label = train_dataset1_y)
xgb_test_dataset1 = xgb.DMatrix(test_dataset1_x, label = test_dataset1_y)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.5,
        'min_child_weight':10,
        'max_depth':3,
        'lambda':11,
        'subsample':0.9,
        'colsample_bytree':1,
#         'colsample_bylevel':0.7,
        'eta': 0.01,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

offline_watchlist = [(xgb_train_dataset1, 'train'), (xgb_test_dataset1, 'val')]
offline_model = xgb.train(cv_params, xgb_train_dataset1, num_boost_round=2950, evals=offline_watchlist)


[0]	train-logloss:0.683918	val-logloss:0.68389
[1]	train-logloss:0.674873	val-logloss:0.674825
[2]	train-logloss:0.666003	val-logloss:0.665939
[3]	train-logloss:0.657307	val-logloss:0.657216
[4]	train-logloss:0.648782	val-logloss:0.648664
[5]	train-logloss:0.640416	val-logloss:0.640278
[6]	train-logloss:0.632207	val-logloss:0.632043
[7]	train-logloss:0.624155	val-logloss:0.623964
[8]	train-logloss:0.616253	val-logloss:0.616036
[9]	train-logloss:0.608495	val-logloss:0.608251
[10]	train-logloss:0.60088	val-logloss:0.600621
[11]	train-logloss:0.593404	val-logloss:0.593126
[12]	train-logloss:0.586064	val-logloss:0.58576
[13]	train-logloss:0.578854	val-logloss:0.578532
[14]	train-logloss:0.571776	val-logloss:0.571433
[15]	train-logloss:0.564823	val-logloss:0.564464
[16]	train-logloss:0.557989	val-logloss:0.557604
[17]	train-logloss:0.551274	val-logloss:0.550871
[18]	train-logloss:0.544677	val-logloss:0.544254
[19]	train-logloss:0.538189	val-logloss:0.537741
[20]	train-logloss:0.531816	val-l

[167]	train-logloss:0.151739	val-logloss:0.148759
[168]	train-logloss:0.150897	val-logloss:0.147901
[169]	train-logloss:0.150066	val-logloss:0.147058
[170]	train-logloss:0.149244	val-logloss:0.146222
[171]	train-logloss:0.148432	val-logloss:0.145397
[172]	train-logloss:0.14763	val-logloss:0.144583
[173]	train-logloss:0.146837	val-logloss:0.143776
[174]	train-logloss:0.146054	val-logloss:0.142981
[175]	train-logloss:0.145281	val-logloss:0.142194
[176]	train-logloss:0.144518	val-logloss:0.141418
[177]	train-logloss:0.143764	val-logloss:0.140651
[178]	train-logloss:0.14302	val-logloss:0.139895
[179]	train-logloss:0.142286	val-logloss:0.139149
[180]	train-logloss:0.141559	val-logloss:0.138411
[181]	train-logloss:0.140842	val-logloss:0.137681
[182]	train-logloss:0.140135	val-logloss:0.136959
[183]	train-logloss:0.139435	val-logloss:0.136248
[184]	train-logloss:0.138745	val-logloss:0.135544
[185]	train-logloss:0.138062	val-logloss:0.134849
[186]	train-logloss:0.137388	val-logloss:0.134162
[1

[332]	train-logloss:0.092681	val-logloss:0.08827
[333]	train-logloss:0.092576	val-logloss:0.088161
[334]	train-logloss:0.092474	val-logloss:0.088054
[335]	train-logloss:0.092373	val-logloss:0.087949
[336]	train-logloss:0.092274	val-logloss:0.087844
[337]	train-logloss:0.092175	val-logloss:0.087743
[338]	train-logloss:0.092079	val-logloss:0.087642
[339]	train-logloss:0.091983	val-logloss:0.087542
[340]	train-logloss:0.091889	val-logloss:0.087445
[341]	train-logloss:0.091797	val-logloss:0.087348
[342]	train-logloss:0.091704	val-logloss:0.087252
[343]	train-logloss:0.091615	val-logloss:0.087158
[344]	train-logloss:0.091526	val-logloss:0.087064
[345]	train-logloss:0.091438	val-logloss:0.086974
[346]	train-logloss:0.091351	val-logloss:0.086885
[347]	train-logloss:0.091267	val-logloss:0.086798
[348]	train-logloss:0.091182	val-logloss:0.086711
[349]	train-logloss:0.091099	val-logloss:0.086626
[350]	train-logloss:0.091016	val-logloss:0.086539
[351]	train-logloss:0.090936	val-logloss:0.086455
[

[497]	train-logloss:0.085544	val-logloss:0.080876
[498]	train-logloss:0.085529	val-logloss:0.080862
[499]	train-logloss:0.085515	val-logloss:0.08085
[500]	train-logloss:0.085501	val-logloss:0.080836
[501]	train-logloss:0.085488	val-logloss:0.080822
[502]	train-logloss:0.085473	val-logloss:0.080806
[503]	train-logloss:0.085458	val-logloss:0.08079
[504]	train-logloss:0.085444	val-logloss:0.080779
[505]	train-logloss:0.085431	val-logloss:0.080766
[506]	train-logloss:0.085418	val-logloss:0.080754
[507]	train-logloss:0.085405	val-logloss:0.080739
[508]	train-logloss:0.085393	val-logloss:0.080728
[509]	train-logloss:0.08538	val-logloss:0.080718
[510]	train-logloss:0.085367	val-logloss:0.080704
[511]	train-logloss:0.085354	val-logloss:0.080692
[512]	train-logloss:0.085341	val-logloss:0.08068
[513]	train-logloss:0.085328	val-logloss:0.080669
[514]	train-logloss:0.085316	val-logloss:0.080657
[515]	train-logloss:0.085304	val-logloss:0.080644
[516]	train-logloss:0.085291	val-logloss:0.080633
[517

[662]	train-logloss:0.084153	val-logloss:0.079676
[663]	train-logloss:0.084147	val-logloss:0.079673
[664]	train-logloss:0.084141	val-logloss:0.079671
[665]	train-logloss:0.084136	val-logloss:0.079668
[666]	train-logloss:0.084131	val-logloss:0.079664
[667]	train-logloss:0.084126	val-logloss:0.079661
[668]	train-logloss:0.084121	val-logloss:0.079658
[669]	train-logloss:0.084115	val-logloss:0.079652
[670]	train-logloss:0.08411	val-logloss:0.079649
[671]	train-logloss:0.084105	val-logloss:0.079646
[672]	train-logloss:0.0841	val-logloss:0.079645
[673]	train-logloss:0.084095	val-logloss:0.07964
[674]	train-logloss:0.084091	val-logloss:0.079637
[675]	train-logloss:0.084085	val-logloss:0.079632
[676]	train-logloss:0.084081	val-logloss:0.079629
[677]	train-logloss:0.084076	val-logloss:0.079627
[678]	train-logloss:0.084071	val-logloss:0.079626
[679]	train-logloss:0.084066	val-logloss:0.079621
[680]	train-logloss:0.084061	val-logloss:0.079618
[681]	train-logloss:0.084056	val-logloss:0.079616
[682

...

[2288]	train-logloss:0.081112	val-logloss:0.078788
[2289]	train-logloss:0.081112	val-logloss:0.078788
[2290]	train-logloss:0.08111	val-logloss:0.078788
[2291]	train-logloss:0.081109	val-logloss:0.078787
[2292]	train-logloss:0.081108	val-logloss:0.078788
[2293]	train-logloss:0.081107	val-logloss:0.078788
[2294]	train-logloss:0.081106	val-logloss:0.078788
[2295]	train-logloss:0.081104	val-logloss:0.078788
[2296]	train-logloss:0.081104	val-logloss:0.078788
[2297]	train-logloss:0.081102	val-logloss:0.078789
[2298]	train-logloss:0.0811	val-logloss:0.078787
[2299]	train-logloss:0.081099	val-logloss:0.078787
[2300]	train-logloss:0.081098	val-logloss:0.078786
[2301]	train-logloss:0.081098	val-logloss:0.078787
[2302]	train-logloss:0.081096	val-logloss:0.078787
[2303]	train-logloss:0.081096	val-logloss:0.078787
[2304]	train-logloss:0.081094	val-logloss:0.078788
[2305]	train-logloss:0.081093	val-logloss:0.078787
[2306]	train-logloss:0.081093	val-logloss:0.078787
[2307]	train-logloss:0.081092	val-

[2450]	train-logloss:0.080935	val-logloss:0.078783
[2451]	train-logloss:0.080934	val-logloss:0.078784
[2452]	train-logloss:0.080933	val-logloss:0.078784
[2453]	train-logloss:0.080933	val-logloss:0.078784
[2454]	train-logloss:0.080932	val-logloss:0.078784
[2455]	train-logloss:0.080931	val-logloss:0.078784
[2456]	train-logloss:0.080929	val-logloss:0.078783
[2457]	train-logloss:0.080928	val-logloss:0.078782
[2458]	train-logloss:0.080927	val-logloss:0.078782
[2459]	train-logloss:0.080926	val-logloss:0.078782
[2460]	train-logloss:0.080924	val-logloss:0.078782
[2461]	train-logloss:0.080923	val-logloss:0.078782
[2462]	train-logloss:0.080922	val-logloss:0.078782
[2463]	train-logloss:0.080921	val-logloss:0.078781
[2464]	train-logloss:0.080919	val-logloss:0.07878
[2465]	train-logloss:0.080918	val-logloss:0.078779
[2466]	train-logloss:0.080918	val-logloss:0.07878
[2467]	train-logloss:0.080917	val-logloss:0.07878
[2468]	train-logloss:0.080917	val-logloss:0.078781
[2469]	train-logloss:0.080915	val-

[2612]	train-logloss:0.080762	val-logloss:0.078771
[2613]	train-logloss:0.080761	val-logloss:0.07877
[2614]	train-logloss:0.08076	val-logloss:0.07877
[2615]	train-logloss:0.080759	val-logloss:0.078771
[2616]	train-logloss:0.080758	val-logloss:0.078771
[2617]	train-logloss:0.080757	val-logloss:0.07877
[2618]	train-logloss:0.080756	val-logloss:0.07877
[2619]	train-logloss:0.080755	val-logloss:0.07877
[2620]	train-logloss:0.080754	val-logloss:0.07877
[2621]	train-logloss:0.080753	val-logloss:0.07877
[2622]	train-logloss:0.080752	val-logloss:0.078771
[2623]	train-logloss:0.080751	val-logloss:0.078771
[2624]	train-logloss:0.080749	val-logloss:0.078771
[2625]	train-logloss:0.080749	val-logloss:0.078771
[2626]	train-logloss:0.080748	val-logloss:0.07877
[2627]	train-logloss:0.080747	val-logloss:0.07877
[2628]	train-logloss:0.080746	val-logloss:0.07877
[2629]	train-logloss:0.080745	val-logloss:0.07877
[2630]	train-logloss:0.080743	val-logloss:0.07877
[2631]	train-logloss:0.080742	val-logloss:0.

[2774]	train-logloss:0.080599	val-logloss:0.078769
[2775]	train-logloss:0.080598	val-logloss:0.078768
[2776]	train-logloss:0.080597	val-logloss:0.078768
[2777]	train-logloss:0.080595	val-logloss:0.078768
[2778]	train-logloss:0.080594	val-logloss:0.078768
[2779]	train-logloss:0.080593	val-logloss:0.078768
[2780]	train-logloss:0.080592	val-logloss:0.078769
[2781]	train-logloss:0.08059	val-logloss:0.078768
[2782]	train-logloss:0.08059	val-logloss:0.078768
[2783]	train-logloss:0.080588	val-logloss:0.078769
[2784]	train-logloss:0.080587	val-logloss:0.078769
[2785]	train-logloss:0.080586	val-logloss:0.078769
[2786]	train-logloss:0.080584	val-logloss:0.078769
[2787]	train-logloss:0.080583	val-logloss:0.078768
[2788]	train-logloss:0.080582	val-logloss:0.078768
[2789]	train-logloss:0.08058	val-logloss:0.078768
[2790]	train-logloss:0.080579	val-logloss:0.078767
[2791]	train-logloss:0.080578	val-logloss:0.078767
[2792]	train-logloss:0.080576	val-logloss:0.078767
[2793]	train-logloss:0.080576	val-

[2936]	train-logloss:0.080436	val-logloss:0.078758
[2937]	train-logloss:0.080435	val-logloss:0.078758
[2938]	train-logloss:0.080433	val-logloss:0.078758
[2939]	train-logloss:0.080432	val-logloss:0.078759
[2940]	train-logloss:0.08043	val-logloss:0.078759
[2941]	train-logloss:0.080429	val-logloss:0.078758
[2942]	train-logloss:0.080429	val-logloss:0.078758
[2943]	train-logloss:0.080427	val-logloss:0.078758
[2944]	train-logloss:0.080426	val-logloss:0.078758
[2945]	train-logloss:0.080425	val-logloss:0.078758
[2946]	train-logloss:0.080425	val-logloss:0.078758
[2947]	train-logloss:0.080424	val-logloss:0.078758
[2948]	train-logloss:0.080422	val-logloss:0.078757
[2949]	train-logloss:0.080421	val-logloss:0.078757


In [14]:
#输出线下训练的xgb模型的重要性
feat_imp = pd.Series(offline_model.get_fscore()).sort_values(ascending=False)
print(feat_imp.head(100))

item_id_converse_smooth_rate              1499
item_sales_level                          1023
user_gender_shop_converse_smooth_rate      975
shop_score_delivery                        917
item_brand_id_converse_smooth_rate         854
hour_converse_smooth_rate                  763
shop_score_description                     682
user_gender_brand_converse_smooth_rate     681
property_number                            674
user_star_level                            647
hour                                       617
shop_score_service                         568
item_price_level                           546
item_brand_id                              535
is_later_clickSameLastCategory             502
later_clickSameLastCategory_count          496
user_age_level                             454
item_pv_level                              448
shop_review_positive_rate                  447
context_page_id                            432
is_highProportion_brand                    424
user_id_conve

In [15]:
#计算线下误差
def calError(col1, col2):
    error = metrics.log_loss(col1, col2)
    return error

xgb_test = xgb.DMatrix(test_dataset1_x)
test_result = offline_model.predict(xgb_test)
dataset1_error = calError(test_dataset1_y.values, test_result)
print('dataset1_error : ', dataset1_error)


dataset1_error :  0.0787567874115


In [16]:
#进行正式模型训练
xgb_train = xgb.DMatrix(train_dataset_x, label=train_dataset_y)
xgb_test = xgb.DMatrix(test_dataset_x)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.5,
        'min_child_weight':10,
        'max_depth':3,
        'lambda':11,
        'subsample':0.9,
        'colsample_bytree':1,
#         'colsample_bylevel':0.7,
        'eta': 0.01,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

watchlist = [(xgb_train,'train')]
model = xgb.train(cv_params, xgb_train, num_boost_round=2950, evals=watchlist)

[0]	train-logloss:0.683904
[1]	train-logloss:0.674842
[2]	train-logloss:0.665962
[3]	train-logloss:0.65725
[4]	train-logloss:0.648708
[5]	train-logloss:0.640331
[6]	train-logloss:0.632108
[7]	train-logloss:0.624042
[8]	train-logloss:0.616127
[9]	train-logloss:0.608358
[10]	train-logloss:0.600734
[11]	train-logloss:0.593248
[12]	train-logloss:0.585897
[13]	train-logloss:0.578675
[14]	train-logloss:0.571583
[15]	train-logloss:0.564617
[16]	train-logloss:0.557773
[17]	train-logloss:0.55105
[18]	train-logloss:0.544442
[19]	train-logloss:0.537945
[20]	train-logloss:0.531561
[21]	train-logloss:0.525284
[22]	train-logloss:0.519114
[23]	train-logloss:0.513048
[24]	train-logloss:0.507085
[25]	train-logloss:0.50122
[26]	train-logloss:0.495452
[27]	train-logloss:0.489778
[28]	train-logloss:0.484194
[29]	train-logloss:0.478701
[30]	train-logloss:0.473296
[31]	train-logloss:0.467979
[32]	train-logloss:0.462749
[33]	train-logloss:0.4576
[34]	train-logloss:0.452533
[35]	train-logloss:0.447545
[36]	tr

[288]	train-logloss:0.097666
[289]	train-logloss:0.097483
[290]	train-logloss:0.097302
[291]	train-logloss:0.097123
[292]	train-logloss:0.096947
[293]	train-logloss:0.096773
[294]	train-logloss:0.096602
[295]	train-logloss:0.096431
[296]	train-logloss:0.096263
[297]	train-logloss:0.096098
[298]	train-logloss:0.095935
[299]	train-logloss:0.095774
[300]	train-logloss:0.095616
[301]	train-logloss:0.095459
[302]	train-logloss:0.095304
[303]	train-logloss:0.095151
[304]	train-logloss:0.095
[305]	train-logloss:0.09485
[306]	train-logloss:0.094703
[307]	train-logloss:0.094558
[308]	train-logloss:0.094416
[309]	train-logloss:0.094275
[310]	train-logloss:0.094136
[311]	train-logloss:0.093998
[312]	train-logloss:0.093862
[313]	train-logloss:0.093729
[314]	train-logloss:0.093597
[315]	train-logloss:0.093466
[316]	train-logloss:0.093337
[317]	train-logloss:0.09321
[318]	train-logloss:0.093085
[319]	train-logloss:0.092961
[320]	train-logloss:0.09284
[321]	train-logloss:0.09272
[322]	train-logloss:0

[572]	train-logloss:0.083623
[573]	train-logloss:0.083615
[574]	train-logloss:0.083607
[575]	train-logloss:0.083599
[576]	train-logloss:0.083591
[577]	train-logloss:0.083584
[578]	train-logloss:0.083577
[579]	train-logloss:0.083569
[580]	train-logloss:0.083562
[581]	train-logloss:0.083555
[582]	train-logloss:0.083548
[583]	train-logloss:0.08354
[584]	train-logloss:0.083533
[585]	train-logloss:0.083525
[586]	train-logloss:0.083519
[587]	train-logloss:0.083512
[588]	train-logloss:0.083505
[589]	train-logloss:0.083497
[590]	train-logloss:0.08349
[591]	train-logloss:0.083483
[592]	train-logloss:0.083476
[593]	train-logloss:0.083469
[594]	train-logloss:0.083463
[595]	train-logloss:0.083456
[596]	train-logloss:0.08345
[597]	train-logloss:0.083444
[598]	train-logloss:0.083438
[599]	train-logloss:0.083431
[600]	train-logloss:0.083425
[601]	train-logloss:0.083419
[602]	train-logloss:0.083413
[603]	train-logloss:0.083406
[604]	train-logloss:0.0834
[605]	train-logloss:0.083394
[606]	train-logloss

[856]	train-logloss:0.082374
[857]	train-logloss:0.08237
[858]	train-logloss:0.082368
[859]	train-logloss:0.082365
[860]	train-logloss:0.082362
[861]	train-logloss:0.08236
[862]	train-logloss:0.082357
[863]	train-logloss:0.082354
[864]	train-logloss:0.082351
[865]	train-logloss:0.082349
[866]	train-logloss:0.082345
[867]	train-logloss:0.082342
[868]	train-logloss:0.08234
[869]	train-logloss:0.082337
[870]	train-logloss:0.082334
[871]	train-logloss:0.082331
[872]	train-logloss:0.082328
[873]	train-logloss:0.082326
[874]	train-logloss:0.082324
[875]	train-logloss:0.082321
[876]	train-logloss:0.082319
[877]	train-logloss:0.082316
[878]	train-logloss:0.082313
[879]	train-logloss:0.08231
[880]	train-logloss:0.082308
[881]	train-logloss:0.082305
[882]	train-logloss:0.082303
[883]	train-logloss:0.0823
[884]	train-logloss:0.082297
[885]	train-logloss:0.082295
[886]	train-logloss:0.082292
[887]	train-logloss:0.082289
[888]	train-logloss:0.082287
[889]	train-logloss:0.082284
[890]	train-logloss:

[1136]	train-logloss:0.081747
[1137]	train-logloss:0.081745
[1138]	train-logloss:0.081743
[1139]	train-logloss:0.081741
[1140]	train-logloss:0.081739
[1141]	train-logloss:0.081738
[1142]	train-logloss:0.081736
[1143]	train-logloss:0.081734
[1144]	train-logloss:0.081733
[1145]	train-logloss:0.08173
[1146]	train-logloss:0.081729
[1147]	train-logloss:0.081727
[1148]	train-logloss:0.081724
[1149]	train-logloss:0.081723
[1150]	train-logloss:0.081721
[1151]	train-logloss:0.08172
[1152]	train-logloss:0.081718
[1153]	train-logloss:0.081717
[1154]	train-logloss:0.081715
[1155]	train-logloss:0.081713
[1156]	train-logloss:0.081711
[1157]	train-logloss:0.081709
[1158]	train-logloss:0.081707
[1159]	train-logloss:0.081705
[1160]	train-logloss:0.081703
[1161]	train-logloss:0.081702
[1162]	train-logloss:0.081701
[1163]	train-logloss:0.081699
[1164]	train-logloss:0.081697
[1165]	train-logloss:0.081695
[1166]	train-logloss:0.081694
[1167]	train-logloss:0.081692
[1168]	train-logloss:0.08169
[1169]	train-

...

[1685]	train-logloss:0.080976
[1686]	train-logloss:0.080975
[1687]	train-logloss:0.080973
[1688]	train-logloss:0.080972
[1689]	train-logloss:0.080971
[1690]	train-logloss:0.080969
[1691]	train-logloss:0.080969
[1692]	train-logloss:0.080968
[1693]	train-logloss:0.080966
[1694]	train-logloss:0.080965
[1695]	train-logloss:0.080963
[1696]	train-logloss:0.080963
[1697]	train-logloss:0.080961
[1698]	train-logloss:0.080961
[1699]	train-logloss:0.080959
[1700]	train-logloss:0.080958
[1701]	train-logloss:0.080957
[1702]	train-logloss:0.080956
[1703]	train-logloss:0.080955
[1704]	train-logloss:0.080954
[1705]	train-logloss:0.080953
[1706]	train-logloss:0.080952
[1707]	train-logloss:0.080951
[1708]	train-logloss:0.080949
[1709]	train-logloss:0.080948
[1710]	train-logloss:0.080947
[1711]	train-logloss:0.080946
[1712]	train-logloss:0.080945
[1713]	train-logloss:0.080943
[1714]	train-logloss:0.080942
[1715]	train-logloss:0.080941
[1716]	train-logloss:0.08094
[1717]	train-logloss:0.080939
[1718]	trai

[1960]	train-logloss:0.080671
[1961]	train-logloss:0.08067
[1962]	train-logloss:0.080669
[1963]	train-logloss:0.080669
[1964]	train-logloss:0.080667
[1965]	train-logloss:0.080666
[1966]	train-logloss:0.080665
[1967]	train-logloss:0.080663
[1968]	train-logloss:0.080662
[1969]	train-logloss:0.080661
[1970]	train-logloss:0.080659
[1971]	train-logloss:0.080657
[1972]	train-logloss:0.080656
[1973]	train-logloss:0.080655
[1974]	train-logloss:0.080654
[1975]	train-logloss:0.080654
[1976]	train-logloss:0.080653
[1977]	train-logloss:0.080652
[1978]	train-logloss:0.080651
[1979]	train-logloss:0.08065
[1980]	train-logloss:0.080648
[1981]	train-logloss:0.080648
[1982]	train-logloss:0.080647
[1983]	train-logloss:0.080646
[1984]	train-logloss:0.080645
[1985]	train-logloss:0.080644
[1986]	train-logloss:0.080643
[1987]	train-logloss:0.080641
[1988]	train-logloss:0.08064
[1989]	train-logloss:0.080639
[1990]	train-logloss:0.080638
[1991]	train-logloss:0.080637
[1992]	train-logloss:0.080636
[1993]	train-

[2234]	train-logloss:0.080393
[2235]	train-logloss:0.080392
[2236]	train-logloss:0.080391
[2237]	train-logloss:0.08039
[2238]	train-logloss:0.080388
[2239]	train-logloss:0.080387
[2240]	train-logloss:0.080386
[2241]	train-logloss:0.080385
[2242]	train-logloss:0.080383
[2243]	train-logloss:0.080382
[2244]	train-logloss:0.08038
[2245]	train-logloss:0.080379
[2246]	train-logloss:0.080378
[2247]	train-logloss:0.080377
[2248]	train-logloss:0.080376
[2249]	train-logloss:0.080374
[2250]	train-logloss:0.080373
[2251]	train-logloss:0.080372
[2252]	train-logloss:0.080371
[2253]	train-logloss:0.08037
[2254]	train-logloss:0.080369
[2255]	train-logloss:0.080368
[2256]	train-logloss:0.080367
[2257]	train-logloss:0.080367
[2258]	train-logloss:0.080366
[2259]	train-logloss:0.080365
[2260]	train-logloss:0.080364
[2261]	train-logloss:0.080363
[2262]	train-logloss:0.080363
[2263]	train-logloss:0.080362
[2264]	train-logloss:0.080361
[2265]	train-logloss:0.08036
[2266]	train-logloss:0.08036
[2267]	train-lo

[2508]	train-logloss:0.080134
[2509]	train-logloss:0.080133
[2510]	train-logloss:0.080132
[2511]	train-logloss:0.08013
[2512]	train-logloss:0.08013
[2513]	train-logloss:0.080129
[2514]	train-logloss:0.080128
[2515]	train-logloss:0.080128
[2516]	train-logloss:0.080127
[2517]	train-logloss:0.080126
[2518]	train-logloss:0.080125
[2519]	train-logloss:0.080124
[2520]	train-logloss:0.080123
[2521]	train-logloss:0.080123
[2522]	train-logloss:0.080122
[2523]	train-logloss:0.080121
[2524]	train-logloss:0.08012
[2525]	train-logloss:0.080119
[2526]	train-logloss:0.080118
[2527]	train-logloss:0.080117
[2528]	train-logloss:0.080116
[2529]	train-logloss:0.080115
[2530]	train-logloss:0.080114
[2531]	train-logloss:0.080113
[2532]	train-logloss:0.080111
[2533]	train-logloss:0.08011
[2534]	train-logloss:0.080109
[2535]	train-logloss:0.080108
[2536]	train-logloss:0.080107
[2537]	train-logloss:0.080106
[2538]	train-logloss:0.080106
[2539]	train-logloss:0.080104
[2540]	train-logloss:0.080103
[2541]	train-l

[2783]	train-logloss:0.079894
[2784]	train-logloss:0.079893
[2785]	train-logloss:0.079892
[2786]	train-logloss:0.079892
[2787]	train-logloss:0.079891
[2788]	train-logloss:0.07989
[2789]	train-logloss:0.079889
[2790]	train-logloss:0.079889
[2791]	train-logloss:0.079888
[2792]	train-logloss:0.079887
[2793]	train-logloss:0.079886
[2794]	train-logloss:0.079885
[2795]	train-logloss:0.079885
[2796]	train-logloss:0.079883
[2797]	train-logloss:0.079882
[2798]	train-logloss:0.079881
[2799]	train-logloss:0.07988
[2800]	train-logloss:0.079879
[2801]	train-logloss:0.079878
[2802]	train-logloss:0.079878
[2803]	train-logloss:0.079877
[2804]	train-logloss:0.079876
[2805]	train-logloss:0.079875
[2806]	train-logloss:0.079874
[2807]	train-logloss:0.079873
[2808]	train-logloss:0.079873
[2809]	train-logloss:0.079872
[2810]	train-logloss:0.079871
[2811]	train-logloss:0.07987
[2812]	train-logloss:0.079869
[2813]	train-logloss:0.079868
[2814]	train-logloss:0.079866
[2815]	train-logloss:0.079866
[2816]	train-

In [17]:
#predict test set
test_df['predicted_score'] = model.predict(xgb_test)


In [18]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../../data/chusai/submit/%s.txt' % fileName, sep=' ', header=True, index=False)

print(test_df[['instance_id', 'predicted_score']].head(10))
exportResult(test_df[['instance_id', 'predicted_score']], 'chusai_xgb_4_22_wen')


        instance_id  predicted_score
0    55144604125888         0.013551
1   221669167755726         0.006938
2   566644865989395         0.019581
3   954943998950521         0.012554
4  1192015136416062         0.033068
5  1239632009133923         0.014525
6  1370758425798224         0.004138
7  1386245957915884         0.002132
8  1587829942816220         0.003735
9  1965296257027358         0.009129


In [19]:
print(len(test_df[['instance_id', 'predicted_score']][test_df.predicted_score > 0.1]))
print(test_df['predicted_score'].mean())
print(len(test_df['predicted_score']))


153
0.014746344648301601
42888
